In [1]:
# Data 1 (same as in the paper)

# Fulfillment center (FC) data
listFC = ['Utah', 'Nevada']
inventoryFC = dict(zip(listFC, [5, 20]))
print(inventoryFC)
probMultiAvailability = dict(zip(listFC, [0.5, 0.2]))
print(probMultiAvailability)

# Customer region data
listRegion = ['Kansas']
demandValue = dict(zip(listRegion, [2*10]))
print(demandValue)
probMultiItem = dict(zip(listRegion, [0.75]))
print(probMultiItem)

# Shipment cost data
costSingle = [[9], [12]]
print(costSingle)

# average number of items in a multi-item order
avgNoMultiItem = 3.0
# we can calculate the multi-item shipping discount $\omega$
shippingDiscount = 1/avgNoMultiItem

# Prepare shipment cost above in dictionary format
costSingleDict = {}
for i in range(len(listFC)):
  for j in range(len(listRegion)):
    costSingleDict[(listFC[i],listRegion[j])] = costSingle[i][j]

print(costSingleDict)

{'Utah': 5, 'Nevada': 20}
{'Utah': 0.5, 'Nevada': 0.2}
{'Kansas': 20}
{'Kansas': 0.75}
[[9], [12]]
{('Utah', 'Kansas'): 9, ('Nevada', 'Kansas'): 12}


In [2]:
# Data 2

# Fulfillment center (FC) data
listFC = ['Delta-BC', 'Brampton-ON', 'Ottawa-ON']
inventoryFC = dict(zip(listFC, [37, 85, 25]))
print(inventoryFC)
probMultiAvailability = dict(zip(listFC, [0.32, 0.45, 0.17]))
print(probMultiAvailability)

# Customer region data
listRegion = ['Toronto', 'Montreal', 'Calgary', 'Vancouver']
demandValue = dict(zip(listRegion, [45, 27, 15, 33]))
print(demandValue)
probMultiItem = dict(zip(listRegion, [0.73, 0.68, 0.54, 0.64]))
print(probMultiItem)

# Shipment cost data
costSingle = [[24.5, 25.5, 18.1, 12.3],
        [13.6, 17.5, 22.8, 23.6],
        [18.1, 14.1, 21.1, 22.8]]

# average number of items in a multi-item order
avgNoMultiItem = 2.5
# we can calculate the multi-item shipping discount $\omega$
shippingDiscount = 1/avgNoMultiItem

# Prepare shipment cost above in dictionary format
costSingleDict = {}
for i in range(len(listFC)):
  for j in range(len(listRegion)):
    costSingleDict[(listFC[i],listRegion[j])] = costSingle[i][j]

print(costSingleDict)

{'Delta-BC': 37, 'Brampton-ON': 85, 'Ottawa-ON': 25}
{'Delta-BC': 0.32, 'Brampton-ON': 0.45, 'Ottawa-ON': 0.17}
{'Toronto': 45, 'Montreal': 27, 'Calgary': 15, 'Vancouver': 33}
{'Toronto': 0.73, 'Montreal': 0.68, 'Calgary': 0.54, 'Vancouver': 0.64}
{('Delta-BC', 'Toronto'): 24.5, ('Delta-BC', 'Montreal'): 25.5, ('Delta-BC', 'Calgary'): 18.1, ('Delta-BC', 'Vancouver'): 12.3, ('Brampton-ON', 'Toronto'): 13.6, ('Brampton-ON', 'Montreal'): 17.5, ('Brampton-ON', 'Calgary'): 22.8, ('Brampton-ON', 'Vancouver'): 23.6, ('Ottawa-ON', 'Toronto'): 18.1, ('Ottawa-ON', 'Montreal'): 14.1, ('Ottawa-ON', 'Calgary'): 21.1, ('Ottawa-ON', 'Vancouver'): 22.8}


In [3]:
from pyomo.environ import *

model = ConcreteModel()
# Variables
model.x = Var(listFC, listRegion, within = NonNegativeReals)
model.y = Var(listFC, listRegion, within = NonNegativeReals)
model.w = Var(listFC, listRegion, within = NonNegativeReals)

model.inventoryOnHand = ConstraintList()
model.demandSingle = ConstraintList()
model.demandMulitiple = ConstraintList()
model.maxMultiShipment = ConstraintList()

# create an object to access to shadow prices. Please note that the codes must be exactly written as
# model.dual = Suffix(direction=Suffix.IMPORT_EXPORT) in order for it to work because it was hard coded in pyomo.
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

In [4]:
# Objective function

obj_expr = sum(costSingleDict[(i,j)]*model.w[(i,j)] + shippingDiscount*costSingleDict[(i,j)]*model.x[(i,j)] \
               + (2*shippingDiscount)*costSingleDict[(i,j)]*model.y[(i,j)] for i in listFC for j in listRegion)
print(obj_expr)
model.OBJ = Objective(expr = obj_expr, sense = minimize)

24.5*w[Delta-BC,Toronto] + 9.8*x[Delta-BC,Toronto] + 19.6*y[Delta-BC,Toronto] + 25.5*w[Delta-BC,Montreal] + 10.200000000000001*x[Delta-BC,Montreal] + 20.400000000000002*y[Delta-BC,Montreal] + 18.1*w[Delta-BC,Calgary] + 7.240000000000001*x[Delta-BC,Calgary] + 14.480000000000002*y[Delta-BC,Calgary] + 12.3*w[Delta-BC,Vancouver] + 4.920000000000001*x[Delta-BC,Vancouver] + 9.840000000000002*y[Delta-BC,Vancouver] + 13.6*w[Brampton-ON,Toronto] + 5.44*x[Brampton-ON,Toronto] + 10.88*y[Brampton-ON,Toronto] + 17.5*w[Brampton-ON,Montreal] + 7.0*x[Brampton-ON,Montreal] + 14.0*y[Brampton-ON,Montreal] + 22.8*w[Brampton-ON,Calgary] + 9.120000000000001*x[Brampton-ON,Calgary] + 18.240000000000002*y[Brampton-ON,Calgary] + 23.6*w[Brampton-ON,Vancouver] + 9.440000000000001*x[Brampton-ON,Vancouver] + 18.880000000000003*y[Brampton-ON,Vancouver] + 18.1*w[Ottawa-ON,Toronto] + 7.240000000000001*x[Ottawa-ON,Toronto] + 14.480000000000002*y[Ottawa-ON,Toronto] + 14.1*w[Ottawa-ON,Montreal] + 5.640000000000001*x[Otta

In [5]:
# Constraints 1 Inventory availability at FC i

for i in listFC:
  const_expr = sum(model.w[(i,j)] + model.x[(i,j)] + model.y[(i,j)] for j in listRegion) <= inventoryFC[i]
  print(const_expr)
  model.inventoryOnHand.add(expr = const_expr)


w[Delta-BC,Toronto] + x[Delta-BC,Toronto] + y[Delta-BC,Toronto] + w[Delta-BC,Montreal] + x[Delta-BC,Montreal] + y[Delta-BC,Montreal] + w[Delta-BC,Calgary] + x[Delta-BC,Calgary] + y[Delta-BC,Calgary] + w[Delta-BC,Vancouver] + x[Delta-BC,Vancouver] + y[Delta-BC,Vancouver]  <=  37
w[Brampton-ON,Toronto] + x[Brampton-ON,Toronto] + y[Brampton-ON,Toronto] + w[Brampton-ON,Montreal] + x[Brampton-ON,Montreal] + y[Brampton-ON,Montreal] + w[Brampton-ON,Calgary] + x[Brampton-ON,Calgary] + y[Brampton-ON,Calgary] + w[Brampton-ON,Vancouver] + x[Brampton-ON,Vancouver] + y[Brampton-ON,Vancouver]  <=  85
w[Ottawa-ON,Toronto] + x[Ottawa-ON,Toronto] + y[Ottawa-ON,Toronto] + w[Ottawa-ON,Montreal] + x[Ottawa-ON,Montreal] + y[Ottawa-ON,Montreal] + w[Ottawa-ON,Calgary] + x[Ottawa-ON,Calgary] + y[Ottawa-ON,Calgary] + w[Ottawa-ON,Vancouver] + x[Ottawa-ON,Vancouver] + y[Ottawa-ON,Vancouver]  <=  25


In [6]:
# Constraints 2 Future demand for single-item order in region j

for j in listRegion:
  const_expr = sum(model.w[(i,j)] for i in listFC) == demandValue[j]*(1-probMultiItem[j])
  print(const_expr)
  model.demandSingle.add(expr = const_expr)

w[Delta-BC,Toronto] + w[Brampton-ON,Toronto] + w[Ottawa-ON,Toronto]  ==  12.15
w[Delta-BC,Montreal] + w[Brampton-ON,Montreal] + w[Ottawa-ON,Montreal]  ==  8.639999999999999
w[Delta-BC,Calgary] + w[Brampton-ON,Calgary] + w[Ottawa-ON,Calgary]  ==  6.8999999999999995
w[Delta-BC,Vancouver] + w[Brampton-ON,Vancouver] + w[Ottawa-ON,Vancouver]  ==  11.879999999999999


In [7]:
# Constraints 3 Future demand for multi-item order in region j

for j in listRegion:
  const_expr = sum(model.x[(i,j)] + model.y[(i,j)] for i in listFC) == demandValue[j]*probMultiItem[j]
  print(const_expr)
  model.demandMulitiple.add(expr = const_expr)

x[Delta-BC,Toronto] + y[Delta-BC,Toronto] + x[Brampton-ON,Toronto] + y[Brampton-ON,Toronto] + x[Ottawa-ON,Toronto] + y[Ottawa-ON,Toronto]  ==  32.85
x[Delta-BC,Montreal] + y[Delta-BC,Montreal] + x[Brampton-ON,Montreal] + y[Brampton-ON,Montreal] + x[Ottawa-ON,Montreal] + y[Ottawa-ON,Montreal]  ==  18.360000000000003
x[Delta-BC,Calgary] + y[Delta-BC,Calgary] + x[Brampton-ON,Calgary] + y[Brampton-ON,Calgary] + x[Ottawa-ON,Calgary] + y[Ottawa-ON,Calgary]  ==  8.100000000000001
x[Delta-BC,Vancouver] + y[Delta-BC,Vancouver] + x[Brampton-ON,Vancouver] + y[Brampton-ON,Vancouver] + x[Ottawa-ON,Vancouver] + y[Ottawa-ON,Vancouver]  ==  21.12


In [8]:
# Constraints #4 Estimated maximum number multi-item shipments from FC i to customer regions j

for i in listFC:
  for j in listRegion:
    const_expr = model.x[(i,j)] <= demandValue[j]*probMultiItem[j]*probMultiAvailability[i]
    print(const_expr)
    model.maxMultiShipment.add(expr = const_expr)

x[Delta-BC,Toronto]  <=  10.512
x[Delta-BC,Montreal]  <=  5.875200000000001
x[Delta-BC,Calgary]  <=  2.5920000000000005
x[Delta-BC,Vancouver]  <=  6.758400000000001
x[Brampton-ON,Toronto]  <=  14.7825
x[Brampton-ON,Montreal]  <=  8.262000000000002
x[Brampton-ON,Calgary]  <=  3.645000000000001
x[Brampton-ON,Vancouver]  <=  9.504000000000001
x[Ottawa-ON,Toronto]  <=  5.5845
x[Ottawa-ON,Montreal]  <=  3.121200000000001
x[Ottawa-ON,Calgary]  <=  1.3770000000000004
x[Ottawa-ON,Vancouver]  <=  3.5904000000000003


In [9]:
# This is commented so that we won't print out unless necessary. You can outcomment to print it.
# model.pprint()

In [10]:
# Solve the model
opt = SolverFactory('glpk')
opt.solve(model)

model.display()

Model unknown

  Variables:
    x : Size=12, Index=x_index
        Key                          : Lower : Value   : Upper : Fixed : Stale : Domain
          ('Brampton-ON', 'Calgary') :     0 :   3.645 :  None : False : False : NonNegativeReals
         ('Brampton-ON', 'Montreal') :     0 :   8.262 :  None : False : False : NonNegativeReals
          ('Brampton-ON', 'Toronto') :     0 : 14.7825 :  None : False : False : NonNegativeReals
        ('Brampton-ON', 'Vancouver') :     0 :   9.504 :  None : False : False : NonNegativeReals
             ('Delta-BC', 'Calgary') :     0 :   2.592 :  None : False : False : NonNegativeReals
            ('Delta-BC', 'Montreal') :     0 :  5.8752 :  None : False : False : NonNegativeReals
             ('Delta-BC', 'Toronto') :     0 :  1.2412 :  None : False : False : NonNegativeReals
           ('Delta-BC', 'Vancouver') :     0 :  6.7584 :  None : False : False : NonNegativeReals
            ('Ottawa-ON', 'Calgary') :     0 :   1.377 :  None : Fals

In [11]:
# Obtain reduced cost for each constraint
model.dual.display()

dual : Direction=Suffix.IMPORT_EXPORT, Datatype=Suffix.FLOAT
    Key                  : Value
      demandMulitiple[1] :                 10.88
      demandMulitiple[2] :                 11.28
      demandMulitiple[3] :                 15.56
      demandMulitiple[4] :                 10.92
         demandSingle[1] :                  13.6
         demandSingle[2] :                  14.1
         demandSingle[3] :                 19.18
         demandSingle[4] :                 13.38
      inventoryOnHand[1] :                 -1.08
      inventoryOnHand[2] :                   0.0
      inventoryOnHand[3] :                   0.0
    maxMultiShipment[10] :                 -5.64
    maxMultiShipment[11] :                 -7.12
    maxMultiShipment[12] :                  -1.8
     maxMultiShipment[1] :                   0.0
     maxMultiShipment[2] : -1.77635683940025e-15
     maxMultiShipment[3] :                 -7.24
     maxMultiShipment[4] :                 -4.92
     maxMultiShipment[5]